In [1]:
import torch.nn as nn
from sklearn.model_selection import train_test_split
import numpy as np
import scipy.spatial as scisp
import pandas as pd
from matplotlib import pyplot as plt
import matplotlib.patches as mpatches

In [70]:
data = pd.read_csv("data2_with_county.csv", index_col = 0)

In [71]:
data = data.drop("Unnamed: 0.1", 1)

In [72]:
append0 = np.vectorize(lambda x : str(x) if len(str(x)) == 5 else "0" + str(x))
data["zip"] = append0(data["zip"])

In [73]:
# data["city scale"].value_counts()
temp = data[data["scale"] == "Total"]
data = temp[temp["level"] == 1]
# temp["city scale"].value_counts()

In [74]:
data = data.loc[:, ["zip", "ESTAB", "city scale", "County Names", "County Fips", "City Lat", "City Lng"]]

In [75]:
zip2vec = np.load("npdata\zip2vec.npy")
indices = np.load("npdata\indices.npy")
zipcodes = np.load("npdata\zipcodes.npy")

In [76]:
zip2idx = {z : i for z, i in zip(zipcodes, indices)}
idx2zip = {i : z for i, z in zip(indices, zipcodes)}

In [77]:
embedding = zip2vec / (np.sum(zip2vec, axis=1, keepdims= True) + 1e-4) 

In [78]:
labels = np.argmax(embedding, axis = 1)

In [79]:
data["output_labels"] = labels

In [80]:
# embedding.mean(axis = 0)
data["output_labels"].value_counts()

11    9394
10    5080
4     4159
0     3671
7     1923
3     1438
1     1147
5      770
2      706
9      410
8      269
6      145
Name: output_labels, dtype: int64

In [81]:
def to_one_hot(idx): 
    out = np.zeros([12, 1])
    out[idx] = 1
    return out

In [82]:
our_data = pd.read_csv('county_alldata.csv')

In [83]:
our_data

,Unnamed: 0,Unnamed: 0.1,State Name,County Name,County Fip,Total Pop,Total Male Pop,Total Femle Pop,% 0-10,% 11-20,...,% 54,% 55,% 56,"% 61,62",% 61,% 62,"% 71,72",% 71,% 72,% 81
0,0,76,Alabama,Autauga County,1001,55514,0.033811,0.031199,0.133804,0.153439,...,0.012899,0.000000,0.000000,0.047906,0.004237,0.043670,0.027435,0.002039,0.025396,0.029152
1,1,171,Alabama,Baldwin County,1003,190790,0.030059,0.028717,0.121678,0.125971,...,0.035146,0.005232,0.030467,0.095336,0.012397,0.082940,0.087399,0.013103,0.074296,0.035739
2,2,266,Alabama,Barbour County,1005,27201,0.027205,0.029227,0.116209,0.119077,...,0.000000,0.000000,0.011171,0.055809,0.000000,0.000000,0.020620,0.001830,0.018791,0.026252
3,3,361,Alabama,Bibb County,1007,22597,0.027747,0.026198,0.113112,0.125503,...,0.008469,0.016548,0.005045,0.042226,0.000000,0.000000,0.020145,0.000000,0.000000,0.041665
4,4,456,Alabama,Blount County,1009,57826,0.031854,0.029848,0.127555,0.135908,...,0.023935,0.007893,0.035346,0.058543,0.000000,0.000000,0.024446,0.004580,0.019865,0.051510
5,5,551,Alabama,Bullock County,1011,10474,0.035898,0.031602,0.126408,0.108173,...,0.012476,0.014656,0.003795,0.074435,0.000028,0.074407,0.000000,0.000000,0.000000,0.023474
6,6,646,Alabama,Butler County,1013,20307,0.032206,0.029251,0.126951,0.122470,...,0.012907,0.007130,0.023671,0.088452,0.000000,0.000000,0.041770,0.005796,0.035974,0.029320
7,7,741,Alabama,Calhoun County,1015,117296,0.031595,0.029302,0.122502,0.132127,...,0.027858,0.002467,0.057179,0.068243,0.003967,0.064276,0.029437,0.000988,0.028450,0.022521
8,8,836,Alabama,Chambers County,1017,34064,0.028769,0.029327,0.115782,0.126527,...,0.000000,0.000000,0.043294,0.085439,0.000000,0.000000,0.026324,0.001642,0.024682,0.025844
9,9,931,Alabama,Cherokee County,1019,26021,0.024480,0.023904,0.104531,0.123362,...,0.000000,0.000000,0.011157,0.041415,0.000465,0.040950,0.021197,0.001330,0.019866,0.031272


In [84]:
data

,zip,ESTAB,city scale,County Names,County Fips,City Lat,City Lng,output_labels
0,06001,81,large,Hartford,9003,41.79071,-72.85384,7
185,06002,63,large,Hartford,9003,41.84255,-72.74059,10
342,06006,1,tiny,Hartford,9003,41.85424,-72.64405,3
350,06010,168,large,Hartford,9003,41.68116,-72.94074,10
616,06011,2,tiny,Hartford,9003,41.67746,-72.94628,3
631,06013,11,medium,Hartford,9003,41.75978,-72.95893,3
673,06016,6,small,Hartford,9003,41.90417,-72.54448,3
697,06018,17,medium,Litchfield,9005,42.02647,-73.30873,4
768,06019,78,large,Hartford,9003,41.85964,-72.90698,7
944,06020,1,tiny,Hartford,9003,41.83455,-72.92862,4


In [85]:
given_data = pd.read_csv('data2_with_county.csv')
uszips = pd.read_csv('our_data/uszips.csv')

,zip,lat,lng,city,state_id,state_name,zcta,parent_zcta,population,density,...,education_college_or_above,labor_force_participation,unemployment_rate,race_white,race_black,race_asian,race_native,race_pacific,race_other,race_multiple


In [47]:
given_data[given_data['City Name'] == 'Franconia']

,Unnamed: 0,Unnamed: 0.1,zip,NAICS.id,NAICS.display-label,RCPSZFE.display-label,ESTAB,City Name,State,level,weight,scale,city scale,County Names,County Fips,City Lat,City Lng
969321,969321,969321,3580,44-45,Retail trade,All establishments,7,Franconia,NH,1,0,Total,small,Grafton,33009,44.16777,-71.70148
969322,969322,969322,3580,44-45,Retail trade,Establishments operated entire year with sales...,1,Franconia,NH,1,2,C,small,Grafton,33009,44.16777,-71.70148
969323,969323,969323,3580,44-45,Retail trade,Establishments operated entire year with sales...,2,Franconia,NH,1,4,B,small,Grafton,33009,44.16777,-71.70148
969324,969324,969324,3580,44-45,Retail trade,Establishments operated entire year with sales...,3,Franconia,NH,1,8,A,small,Grafton,33009,44.16777,-71.70148
969325,969325,969325,3580,444,Building material and garden equipment and sup...,All establishments,1,Franconia,NH,2,0,Total,small,Grafton,33009,44.16777,-71.70148
969326,969326,969326,3580,444,Building material and garden equipment and sup...,Establishments operated entire year with sales...,1,Franconia,NH,2,2,C,small,Grafton,33009,44.16777,-71.70148
969327,969327,969327,3580,4441,Building material and supplies dealers,All establishments,1,Franconia,NH,3,0,Total,small,Grafton,33009,44.16777,-71.70148
969328,969328,969328,3580,4441,Building material and supplies dealers,Establishments operated entire year with sales...,1,Franconia,NH,3,2,C,small,Grafton,33009,44.16777,-71.70148
969329,969329,969329,3580,44413,Hardware stores,All establishments,1,Franconia,NH,4,0,Total,small,Grafton,33009,44.16777,-71.70148
969330,969330,969330,3580,44413,Hardware stores,Establishments operated entire year with sales...,1,Franconia,NH,4,2,C,small,Grafton,33009,44.16777,-71.70148


In [86]:
def add_total_pop(fip_num):
    result = our_data[our_data['County Fip'] == fip_num]['Total Pop']
    if (len(result)):
        return result.tolist()[0]
    else:
        print(fip_num)
        return -1
data['Total Pop'] = np.vectorize(lambda x : add_total_pop(x))(data['County Fips'])

-2147483648
-2147483648
-2147483648
-2147483648
-2147483648
-2147483648
-2147483648
-2147483648
-2147483648
-2147483648
-2147483648
-2147483648
-2147483648
-2147483648
-2147483648
-2147483648
-2147483648
-2147483648
-2147483648
-2147483648
-2147483648
-2147483648
-2147483648
-2147483648
-2147483648
-2147483648
-2147483648
-2147483648
-2147483648
-2147483648


In [87]:
data = data[data['County Fips'] != -2147483648]

In [92]:
data['Total Pop'] = np.vectorize(lambda x : add_total_pop(x))(data['County Fips'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [90]:
data['Median Income'] = np.vectorize(lambda x : uszips[uszips['county_fips'] == x]['income_household_median'].tolist()[0])(data['County Fips'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [97]:
data['% college or higher'] = np.vectorize(lambda x : our_data[our_data['County Fip'] == x]['% college or higher'].tolist()[0])(data['County Fips'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [98]:
data['% high school only'] = np.vectorize(lambda x : our_data[our_data['County Fip'] == x]['% high school only'].tolist()[0])(data['County Fips'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


In [99]:
data['% 81+'] = np.vectorize(lambda x : our_data[our_data['County Fip'] == x]['% 81+'].tolist()[0])(data['County Fips'])
data['% 21-30'] = np.vectorize(lambda x : our_data[our_data['County Fip'] == x]['% 21-30'].tolist()[0])(data['County Fips'])
data['% 31-40'] = np.vectorize(lambda x : our_data[our_data['County Fip'] == x]['% 31-40'].tolist()[0])(data['County Fips'])
data['% 41-50'] = np.vectorize(lambda x : our_data[our_data['County Fip'] == x]['% 41-50'].tolist()[0])(data['County Fips'])
data['% 51-60'] = np.vectorize(lambda x : our_data[our_data['County Fip'] == x]['% 51-60'].tolist()[0])(data['County Fips'])
data['% 61-70'] = np.vectorize(lambda x : our_data[our_data['County Fip'] == x]['% 61-70'].tolist()[0])(data['County Fips'])
data['% 71-80'] = np.vectorize(lambda x : our_data[our_data['County Fip'] == x]['% 71-80'].tolist()[0])(data['County Fips'])

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.

In [103]:
data.describe()

,ESTAB,County Fips,City Lat,City Lng,output_labels,Total Pop,Median Income,% college or higher,% high school only,% 81+,% 21-30,% 31-40,% 41-50,% 51-60,% 61-70,% 71-80
count,29082.000000,29082.000000,29082.000000,29082.000000,29082.000000,2.908200e+04,23855.000000,29082.000000,29082.000000,29082.000000,29082.000000,29082.000000,29082.000000,29082.000000,29082.000000,29082.000000
mean,35.150265,28907.584588,38.545703,-90.106933,6.929682,5.328696e+05,61357.356571,0.269975,0.310633,0.041612,0.130039,0.120098,0.133464,0.144528,0.110796,0.063176
std,58.223428,15636.713837,5.293974,15.157875,4.145240,1.283432e+06,27613.801041,0.112591,0.080206,0.013363,0.032560,0.017383,0.013762,0.016439,0.022528,0.018222
min,1.000000,1001.000000,19.070260,-176.631390,0.000000,9.000000e+01,2499.000000,0.054000,0.055000,0.005333,0.037313,0.062906,0.063995,0.038813,0.027460,0.014783
25%,2.000000,17077.000000,34.865555,-95.804107,4.000000,3.291500e+04,43427.000000,0.182000,0.253000,0.032609,0.109101,0.108793,0.124653,0.133998,0.095109,0.050559
50%,9.000000,28080.000000,39.320025,-86.763075,9.000000,1.115800e+05,54706.000000,0.245500,0.308000,0.040104,0.124185,0.118476,0.133800,0.145162,0.108511,0.060559
75%,43.000000,42071.000000,41.945715,-79.668685,11.000000,4.901335e+05,71409.000000,0.337000,0.370000,0.048479,0.145104,0.130393,0.142521,0.154521,0.122500,0.072981
max,931.000000,56045.000000,71.273740,-67.017920,11.000000,9.962789e+06,220313.000000,0.785000,0.556000,0.155556,0.375351,0.221711,0.243910,0.238942,0.286105,0.239372


In [104]:
data.to_csv('train_data.csv')